In [1]:
import pandas as pd
from collections import defaultdict

/var/folders/jb/htz2wz0x3s93z17kl82cw22c0000gn/T/ipykernel_5239/4104312911.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
event_names = [
    "200 Medley Relay",
    "200 Freestyle",
    "100 Butterfly",
    "50 Freestyle",
    "100 Freestyle",
    "100 Breaststroke",
    "100 Backstroke",
    "500 Freestyle",
    "200 Freestyle Relay",
    "200 Individual Medley",
    "400 Freestyle Relay"
]

In [3]:
all_teams = []
all_teams_info = []
class_1_teams = []

with open("./class_assignments.txt", "r") as class_assignments:
    lines = class_assignments.readlines()

    for l in lines:
        as_list = l.split("\t")
        all_teams.append(as_list[0])
        all_teams_info.append({"name": as_list[0], "location": as_list[1], "class": as_list[2]})

        if int(as_list[2])==1:
            class_1_teams.append(as_list[0])

len(class_1_teams)

89

In [5]:
big_fat_ugly_and_dirty = pd.read_csv("./Girls Swimming Export (1).csv")
bfuad_columns = list(big_fat_ugly_and_dirty.columns)

In [6]:
# look and see the ugliness

big_fat_ugly_and_dirty.head()

,Unnamed: 0,Unnamed: 1,200 Medley Relay,Unnamed: 3,Unnamed: 4,Unnamed: 5,200 Freestyle,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,100 Breaststroke,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,400 Freestyle Relay,Unnamed: 51
0,1.0,1:46.01,St. Joseph's Academy,NaN,1.0,1:51.09,Helena Tietjen,Pembroke Hill,NaN,1.0,...,NaN,1.0,1:03.18,Reagan Graybill,Staley,NaN,1.0,3:31.11,Park Hill South,NaN
1,2.0,1:47.94,Park Hill South,NaN,2.0,1:51.93,Alexis Cook,Parkway South,NaN,2.0,...,NaN,2.0,1:05.35,Ashlyn Canale,Visitation Academy,NaN,2.0,3:32.11,St. Joseph's Academy,NaN
2,3.0,1:49.95,Eureka,NaN,3.0,1:52.14,Anna-Grace Guenther,St. Joseph's Academy,NaN,3.0,...,NaN,3.0,1:06.10,Charlotte Brown,Villa Duchesne,NaN,3.0,3:36.29,Central (Cape Girardeau),NaN
3,4.0,1:50.94,Kirkwood,NaN,4.0,1:52.56,Avery Cunneen,St. Joseph's Academy,NaN,4.0,...,NaN,4.0,1:06.52,Payton Robic,St. Joseph's Academy,NaN,4.0,3:37.93,Eureka,NaN
4,5.0,1:51.28,Staley,NaN,5.0,1:53.68,Caitlin Hartwig,Liberty,NaN,5.0,...,NaN,5.0,1:06.90,Helena Tietjen,Pembroke Hill,NaN,5.0,3:38.45,Kirkwood,NaN


In [7]:
# extracting event info

all_events = []
all_class_1_events = []
all_class_2_events = []

for i in range(len(bfuad_columns)):
    col = bfuad_columns[i]
    
    if col in event_names:
        event = {"event_name": col, "times": []}
        event1 = {"event_name": col, "times": []}
        event2 = {"event_name": col, "times": []}

        class_1_adjusted_place = 1
        class_2_adjusted_place = 1
        for j in range(len(big_fat_ugly_and_dirty[col].dropna())):
            name = big_fat_ugly_and_dirty[col].dropna()[j]
            team_name = big_fat_ugly_and_dirty[col].dropna()[j]
            time = big_fat_ugly_and_dirty[bfuad_columns[i-1]].dropna()[j]
            
            if "Relay" not in col:
                name = big_fat_ugly_and_dirty[col].dropna()[j]
                team_name = big_fat_ugly_and_dirty[bfuad_columns[i+1]].dropna()[j]

            event["times"].append({
                "team": team_name, "name": name, "time": time,
                "place": int(big_fat_ugly_and_dirty[bfuad_columns[i-2]].dropna()[j]),
            })

            if team_name in class_1_teams:
                event1["times"].append({
                    "team": team_name, "name": name, "time": time,
                    "place": class_1_adjusted_place,
                })

                class_1_adjusted_place += 1
            else:
                event2["times"].append({
                    "team": team_name, "name": name, "time": time,
                    "place": class_2_adjusted_place,
                })

                class_2_adjusted_place += 1
        
        all_events.append(event)
        all_class_1_events.append(event1)
        all_class_2_events.append(event2)

In [8]:
all_class_1_events

[{'event_name': '200 Medley Relay',
  'times': [{'team': "St. Joseph's Academy",
    'name': "St. Joseph's Academy",
    'time': '1:46.01',
    'place': 1},
   {'team': 'Pembroke Hill',
    'name': 'Pembroke Hill',
    'time': '1:51.63',
    'place': 2},
   {'team': 'Visitation Academy',
    'name': 'Visitation Academy',
    'time': '1:52.04',
    'place': 3},
   {'team': 'Villa Duchesne',
    'name': 'Villa Duchesne',
    'time': '1:53.36',
    'place': 4},
   {'team': 'Clayton', 'name': 'Clayton', 'time': '1:53.81', 'place': 5},
   {'team': 'John Burroughs',
    'name': 'John Burroughs',
    'time': '1:54.14',
    'place': 6},
   {'team': 'Central (Cape Girardeau)',
    'name': 'Central (Cape Girardeau)',
    'time': '1:54.16',
    'place': 7},
   {'team': 'Parkway West',
    'name': 'Parkway West',
    'time': '1:55.81',
    'place': 8},
   {'team': 'West Plains',
    'name': 'West Plains',
    'time': '1:56.25',
    'place': 9},
   {'team': 'Monett', 'name': 'Monett', 'time': '1:58

In [9]:
copy_events = all_class_1_events

# sorting people/teams into their fastest events (now I only care about class 1)
# also limits total num
events = dict(zip(event_names, [[] for _ in range(len(event_names))]))
people_events = defaultdict()

finished_events = []

while True:
    for i, event in enumerate(copy_events):
        if len(events[event["event_name"]])==16:
            if event["event_name"] not in finished_events:
                finished_events.append(event["event_name"])

            if len(finished_events)==len(event_names):
                break
            continue

        if "Relay" in event["event_name"]:
            try:
                events[event["event_name"]].append(event["times"][0])
            except:
                break
        else:
            try:
                people_events[event["times"][0]["name"]] += 1
            except KeyError:
                people_events[event["times"][0]["name"]] = 1
            except:
                break

            if people_events[event["times"][0]["name"]] < 3:
                try:
                    events[event["event_name"]].append(event["times"][0])
                except:
                    break
        
        try:
            copy_events[i]["times"].pop(0)
        except:
            break
    
    if len(finished_events)==len(event_names):
        break

In [18]:
for i, e in enumerate(events['200 Freestyle']):
    print("Place:",i+1,"info:",e)

Place: 1 info: {'team': 'Pembroke Hill', 'name': 'Helena Tietjen', 'time': '1:51.09', 'place': 1, 'event': '200 Freestyle'}
Place: 2 info: {'team': "St. Joseph's Academy", 'name': 'Anna-Grace Guenther', 'time': '1:52.14', 'place': 2, 'event': '200 Freestyle'}
Place: 3 info: {'team': "St. Joseph's Academy", 'name': 'Avery Cunneen', 'time': '1:52.56', 'place': 3, 'event': '200 Freestyle'}
Place: 4 info: {'team': 'Pembroke Hill', 'name': 'Kaya Alvegaard-Struble', 'time': '1:56.26', 'place': 4, 'event': '200 Freestyle'}
Place: 5 info: {'team': 'Parkway West', 'name': 'Sydney Matheny', 'time': '1:57.61', 'place': 5, 'event': '200 Freestyle'}
Place: 6 info: {'team': 'Father Tolton Regional Catholic', 'name': 'Madison Arroyo', 'time': '1:57.65', 'place': 6, 'event': '200 Freestyle'}
Place: 7 info: {'team': 'Webster Groves', 'name': 'Quinn Porter', 'time': '1:57.68', 'place': 7, 'event': '200 Freestyle'}
Place: 8 info: {'team': 'Central (Cape Girardeau)', 'name': 'Kennedy Ringwald', 'time': '1

In [20]:
for i, e in enumerate(events['100 Breaststroke']):
    print("Place:",i+1,"info:",e)

Place: 1 info: {'team': 'Visitation Academy', 'name': 'Ashlyn Canale', 'time': '1:05.35', 'place': 1, 'event': '100 Breaststroke'}
Place: 2 info: {'team': 'Villa Duchesne', 'name': 'Charlotte Brown', 'time': '1:06.10', 'place': 2, 'event': '100 Breaststroke'}
Place: 3 info: {'team': "St. Joseph's Academy", 'name': 'Payton Robic', 'time': '1:06.52', 'place': 3, 'event': '100 Breaststroke'}
Place: 4 info: {'team': "St. Joseph's Academy", 'name': 'Vienna Schindler', 'time': '1:07.38', 'place': 4, 'event': '100 Breaststroke'}
Place: 5 info: {'team': 'John Burroughs', 'name': 'Jacqueline Hu', 'time': '1:07.48', 'place': 5, 'event': '100 Breaststroke'}
Place: 6 info: {'team': 'St. Charles West', 'name': 'Stella Garrison', 'time': '1:07.49', 'place': 6, 'event': '100 Breaststroke'}
Place: 7 info: {'team': 'Notre Dame de Sion', 'name': 'Finley Glennon', 'time': '1:09.02', 'place': 7, 'event': '100 Breaststroke'}
Place: 8 info: {'team': 'Clayton', 'name': 'Ema Stojicevic', 'time': '1:09.16', 'p

In [10]:
for i, e in enumerate(events['100 Freestyle']):
    print("Place:",i+1,"info:",e)

Place: 1 info: {'team': 'Central (Cape Girardeau)', 'name': 'Karsynne McAlister', 'time': '51.93', 'place': 2}
Place: 2 info: {'team': 'Central (Cape Girardeau)', 'name': 'Sydney Ringwald', 'time': '52.20', 'place': 3}
Place: 3 info: {'team': 'St. Michael the Archangel', 'name': 'Avery Frick', 'time': '53.19', 'place': 6}
Place: 4 info: {'team': 'West Plains', 'name': 'Roslyn Fiorino', 'time': '53.75', 'place': 7}
Place: 5 info: {'team': "St. Joseph's Academy", 'name': 'Eleanor Green', 'time': '54.15', 'place': 9}
Place: 6 info: {'team': 'Parkway Central', 'name': 'Cecilia Tremont', 'time': '54.71', 'place': 11}
Place: 7 info: {'team': 'Carl Junction', 'name': 'Elyanna Dogotch', 'time': '55.12', 'place': 16}
Place: 8 info: {'team': 'John Burroughs', 'name': 'Madeleine Williams', 'time': '55.14', 'place': 17}
Place: 9 info: {'team': 'Father Tolton Regional Catholic', 'name': 'Kayla Perrone', 'time': '55.34', 'place': 19}
Place: 10 info: {'team': 'Central (Cape Girardeau)', 'name': 'Isab

In [11]:
for i, e in enumerate(events['50 Freestyle']):
    print("Place:",i+1,"info:",e)

Place: 1 info: {'team': 'Central (Cape Girardeau)', 'name': 'Karsynne McAlister', 'time': 23.54, 'place': 1}
Place: 2 info: {'team': 'Central (Cape Girardeau)', 'name': 'Sydney Ringwald', 'time': 24.09, 'place': 3}
Place: 3 info: {'team': 'St. Michael the Archangel', 'name': 'Avery Frick', 'time': 24.14, 'place': 4}
Place: 4 info: {'team': 'Notre Dame de Sion', 'name': 'Finley Glennon', 'time': 24.42, 'place': 5}
Place: 5 info: {'team': 'Ursuline Academy', 'name': 'Kennedy Chrun', 'time': 24.59, 'place': 6}
Place: 6 info: {'team': 'West Plains', 'name': 'Roslyn Fiorino', 'time': 24.89, 'place': 7}
Place: 7 info: {'team': "St. Teresa's Academy", 'name': 'Ava Fugate', 'time': 24.92, 'place': 8}
Place: 8 info: {'team': 'Carl Junction', 'name': 'Elyanna Dogotch', 'time': 25.03, 'place': 9}
Place: 9 info: {'team': 'Parkway Central', 'name': 'Cecilia Tremont', 'time': 25.04, 'place': 10}
Place: 10 info: {'team': "St. Joseph's Academy", 'name': 'Claire Chumley', 'time': 25.13, 'place': 11}
Pl

In [23]:
# calculate team scores

team_scores = dict()

scoring = [20,17,16,15,14,13,12,11,9,7,6,5,4,3,2,1]
relay_scoring = [40,34,32,30,28,26,24,22,18,14,12,10,8,6,4,2]

pc_scores = {}
pc_winning_events = []

st_joe_scores = {}
st_joe_events = []

for event in events:
    for idx, time in enumerate(events[event]):
        # if idx >= 16:
        #     break
        score_2_add = scoring[idx]

        if "Relay" in event:
            score_2_add = relay_scoring[idx]

        if "Parkway Central" == time['team']:
            try:
                pc_scores[events[event][idx]['name']] += score_2_add
            except:
                pc_scores[events[event][idx]['name']] = score_2_add
                
            pc_winning_events.append(events[event][idx])

        elif "St. Joseph's Academy" == time['team']:
            try:
                st_joe_scores[events[event][idx]['name']] += score_2_add
            except:
                st_joe_scores[events[event][idx]['name']] = score_2_add
                
            st_joe_events.append(events[event][idx])

        try:
            team_scores[time['team']] += score_2_add
        except:
            team_scores[time['team']] = score_2_add

In [24]:
for event in st_joe_events:
    print(event)

{'team': "St. Joseph's Academy", 'name': "St. Joseph's Academy", 'time': '1:46.01', 'place': 1, 'event': '200 Medley Relay'}
{'team': "St. Joseph's Academy", 'name': 'Anna-Grace Guenther', 'time': '1:52.14', 'place': 2, 'event': '200 Freestyle'}
{'team': "St. Joseph's Academy", 'name': 'Avery Cunneen', 'time': '1:52.56', 'place': 3, 'event': '200 Freestyle'}
{'team': "St. Joseph's Academy", 'name': 'Payton Robic', 'time': '58.39', 'place': 4, 'event': '100 Butterfly'}
{'team': "St. Joseph's Academy", 'name': 'Maddie Riley', 'time': '1:01.71', 'place': 13, 'event': '100 Butterfly'}
{'team': "St. Joseph's Academy", 'name': 'Claire Chumley', 'time': 25.13, 'place': 10, 'event': '50 Freestyle'}
{'team': "St. Joseph's Academy", 'name': 'Eleanor Green', 'time': 25.46, 'place': 15, 'event': '50 Freestyle'}
{'team': "St. Joseph's Academy", 'name': 'Eleanor Green', 'time': '54.15', 'place': 5, 'event': '100 Freestyle'}
{'team': "St. Joseph's Academy", 'name': 'Payton Robic', 'time': '1:06.52', 

In [13]:
for event in pc_winning_events:
    print(event)

{'team': 'Parkway Central', 'name': 'Parkway Central', 'time': '1:59.35', 'place': 13}
{'team': 'Parkway Central', 'name': 'Reese Anderson', 'time': '57.71', 'place': 3}
{'team': 'Parkway Central', 'name': 'Serena Huang', 'time': '59.02', 'place': 5}
{'team': 'Parkway Central', 'name': 'Cecilia Tremont', 'time': 25.04, 'place': 10}
{'team': 'Parkway Central', 'name': 'Kenadie Johnson', 'time': 25.43, 'place': 14}
{'team': 'Parkway Central', 'name': 'Cecilia Tremont', 'time': '54.71', 'place': 11}
{'team': 'Parkway Central', 'name': 'Gabrielle Salmans', 'time': '56.89', 'place': 34}
{'team': 'Parkway Central', 'name': 'Serena Huang', 'time': '1:09.57', 'place': 11}
{'team': 'Parkway Central', 'name': 'Reese Anderson', 'time': '5:12.36', 'place': 5}
{'team': 'Parkway Central', 'name': 'Parkway Central', 'time': '1:41.69', 'place': 5}
{'team': 'Parkway Central', 'name': 'Parkway Central', 'time': '3:41.55', 'place': 3}


In [14]:
sorted(team_scores.items(), key=lambda x:x[1], reverse=True)[0:10]

[("St. Joseph's Academy", 302),
 ('Central (Cape Girardeau)', 204),
 ('Pembroke Hill', 165),
 ('Clayton', 153),
 ('Parkway Central', 149),
 ('Visitation Academy', 127),
 ('Villa Duchesne', 125),
 ("St. Teresa's Academy", 104),
 ('West Plains', 100),
 ('John Burroughs', 89)]

In [15]:
# scoring people because that's funnnnnnn!!!!!!!!!!

people_scores = dict()

for e in events:
    event = events[e]
    for i, info in enumerate(event):
        info['event'] = e
        info['place'] = i+1
        if info['name'] in class_1_teams:
            continue
        if info['name'] in people_scores:
            people_scores[info['name']]['events'].append(info)
            people_scores[info['name']]['score'] += scoring[i]
        else:
            people_scores[info['name']] = dict()
            people_scores[info['name']]['events'] = [info]
            people_scores[info['name']]['score'] = scoring[i]

In [16]:
people_scores['Reese Anderson']

{'events': [{'team': 'Parkway Central',
   'name': 'Reese Anderson',
   'time': '57.71',
   'place': 3,
   'event': '100 Butterfly'},
  {'team': 'Parkway Central',
   'name': 'Reese Anderson',
   'time': '5:12.36',
   'place': 4,
   'event': '500 Freestyle'}],
 'score': 31}